In [12]:
import torch
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, MegaForMaskedLM
import torchinfo
torch.__version__

'2.2.0'

In [2]:
device = torch.device("mps")
device

device(type='mps')

In [3]:
tokenizer = AutoTokenizer.from_pretrained("mnaylor/mega-base-wikitext")
model1 = MegaForMaskedLM.from_pretrained("mnaylor/mega-base-wikitext")
model1.to(device)

Some weights of MegaForMaskedLM were not initialized from the model checkpoint at mnaylor/mega-base-wikitext and are newly initialized: ['mlm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MegaForMaskedLM(
  (mega): MegaModel(
    (embedding_layer): MegaEmbeddings(
      (word_embeddings): Embedding(50265, 128, padding_idx=1)
    )
    (layers): ModuleList(
      (0-3): 4 x MegaBlock(
        (mega_layer): MegaMovingAverageGatedAttention(
          (activation): SiLU()
          (dropout): MegaDropout()
          (hidden_dropout): MegaDropout()
          (attention_dropout): MegaDropout()
          (norm): MegaSequenceNorm(
            (norm): MegaScaleNorm()
          )
          (ema_gate): MegaMultiDimensionDampedEma()
          (v_proj): Linear(in_features=128, out_features=256, bias=True)
          (mx_proj): Linear(in_features=128, out_features=576, bias=True)
          (h_proj): Linear(in_features=256, out_features=128, bias=True)
          (rel_pos_bias): MegaRotaryRelativePositionalBias()
          (softmax): Softmax(dim=-1)
        )
        (nffn): MegaNormalizedFeedForwardNetwork(
          (activation): SiLU()
          (dropout): MegaDropout()
          (hi

In [4]:
torchinfo.summary(model1)

Layer (type:depth-idx)                                            Param #
MegaForMaskedLM                                                   --
├─MegaModel: 1-1                                                  --
│    └─MegaEmbeddings: 2-1                                        --
│    │    └─Embedding: 3-1                                        6,433,920
│    └─ModuleList: 2-2                                            --
│    │    └─MegaBlock: 3-2                                        223,042
│    │    └─MegaBlock: 3-3                                        223,042
│    │    └─MegaBlock: 3-4                                        223,042
│    │    └─MegaBlock: 3-5                                        223,042
├─Linear: 1-2                                                     6,484,185
├─Dropout: 1-3                                                    --
Total params: 13,810,273
Trainable params: 13,810,273
Non-trainable params: 0

In [5]:
tokenizer

RobertaTokenizerFast(name_or_path='mnaylor/mega-base-wikitext', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

In [21]:
data_check_test = load_dataset(
    "csv", data_files="data/tweets.csv"
    )
print(data_check_test['train'][0])

# Dataset only have training data, hence split it into train/test
data_check_test = data_check_test['train'].train_test_split(test_size=0.5)

{'idx': 0, 'count': 3, 'hate_speech': 0, 'offensive_language': 0, 'neither': 3, 'class': 2, 'tweet': "!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out..."}


In [16]:
def tokenize_func(examples):
    #  padding: 'max_length': pad to a length specified by the max_length argument or the
    #  maximum length accepted by the model if no max_length is provided (max_length=None).
    #  Padding will still be applied if you only provide a single sequence. [from documentation]


    #  truncation: True or 'longest_first': truncate to a maximum length specified
    #  by the max_length argument or the maximum length accepted by the model if
    #  no max_length is provided (max_length=None). This will truncate token by
    #  token, removing a token from the longest sequence in the pair until the
    #  proper length is reached. [from documentation]
    return tokenizer(examples, padding="max_length", truncation=True)

In [22]:
tokenized_data = data_check_test.map(tokenize_func, batched=True)
tokenized_data = tokenized_data.remove_columns([""])


Map:   0%|          | 0/12391 [00:00<?, ? examples/s]

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).